In [1]:
%defaultDatasource jdbc:h2:mem:db

# Criação das tabelas Country e CasosCovid

In [2]:
CREATE TABLE Country (
    iso_code VARCHAR(8) NOT NULL,
    name VARCHAR(50),
    continent VARCHAR(50),
    population DOUBLE,
    population_density DOUBLE,
    median_age DOUBLE,
    aged_65_older DOUBLE,
    aged_70_older DOUBLE,
    gdp_per_capita DOUBLE,
    extreme_poverty DOUBLE,
    cardiovasc_death_rate DOUBLE,
    diabetes_prevalence DOUBLE,
    female_smokers DOUBLE,
    male_smokers DOUBLE,
    handwashing_facilities DOUBLE,
    hospital_beds_per_thousand DOUBLE,
    life_expectancy DOUBLE,
    human_development_index DOUBLE,
    PRIMARY KEY (iso_code)
) AS SELECT
    iso_code,
    location,
    continent,
    population,
    population_density,
    median_age,
    aged_65_older,
    aged_70_older,
    gdp_per_capita,
    extreme_poverty,
    cardiovasc_death_rate,
    diabetes_prevalence,
    female_smokers,
    male_smokers,
    handwashing_facilities,
    hospital_beds_per_thousand,
    life_expectancy,
    human_development_index
    FROM CSVREAD('owid-country-data.csv');

CREATE TABLE CasosCovid (
    iso_code VARCHAR(8) NOT NULL,
    date DATE NOT NULL,
    total_cases DOUBLE,
    new_cases DOUBLE, 
    total_deaths DOUBLE,
    new_deaths DOUBLE,
    total_tests DOUBLE,
    FOREIGN KEY (iso_code) REFERENCES Country (iso_code)
) AS SELECT
    iso_code,
    date,
    total_cases,
    new_cases,
    total_deaths,
    new_deaths,
    total_tests
    FROM CSVREAD('owid-covid-data.csv')

In [27]:
SELECT * FROM Country;
SELECT * FROM CasosCovid;

# Algumas queries úteis para analise rápida

## 1) Ultimos números de mortes em Taiwan

In [23]:
SELECT c.name, co.total_deaths, co.date
    FROM Country c, CasosCovid co
    WHERE c.iso_code = co.iso_code and c.iso_code = 'TWN'
    ORDER BY date DESC
    LIMIT 1
    

## 2) Quantidade de fumantes de cada país

In [4]:
SELECT name, female_smokers + male_smokers as Fumantes
    FROM Country
    ORDER BY Fumantes DESC

## 3) Quantidade de pessoas idosas de cada país

In [6]:
SELECT name, aged_65_older + aged_70_older as Idosos
    FROM Country
    ORDER BY Idosos DESC

## 4) Número de mortes por COVID por ordem de IDH

In [13]:
SELECT c.name, co.total_deaths, c.human_development_index, co.date
    FROM Country c, CasosCovid co
    WHERE c.iso_code = co.iso_code
    ORDER BY co.date DESC, c.human_development_index DESC
    LIMIT 210

## 5) Análise da espectativa de vida com mortes por COVID

In [16]:
SELECT c.name, c.life_expectancy, co.total_deaths, co.date
    FROM Country c, CasosCovid co
    WHERE c.iso_code = co.iso_code
    ORDER BY co.date DESC, c.life_expectancy DESC
    LIMIT 210